In [2]:
from bs4 import BeautifulSoup
import requests
import os

import pandas as pd
from pandas import Series,DataFrame


import csv
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series,DataFrame
import urllib
import pickle

# now we want to collect email addresses
# Add javascript with selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import time
from datetime import datetime

from bs4 import BeautifulSoup
import requests
import pandas as pd
from urllib.request import Request, urlopen
import re

In [ ]:
# function to extract the scores from the webpage
def extract_scores(content):
    temp_dict = {}

    # first we look for main esg score
    ele = content.find("h3", {"class": "Heading Heading--m"})

    try:
        print(ele.text.split(":")[1].strip())
        temp_dict["main_score"] = ele.text.split(":")[1].strip()
    except:
        temp_dict["main_score"] = ""
    # then we look for sub esg scores
    env = content.find_all("div", {"class": "Grid-item"})

    # loop through all divs
    for el in env:
        # loop through all div table items
        table_ele = el.find_all("div", {"class": "table-item"})

        for sub_note in table_ele:
            # get names of the sub category
            name = sub_note.find("div", {"class": "table-align-left"})

            # get score of the sub category
            val = sub_note.find("div", {"class": "table-align-right"})
            temp_dict[name.text] = val.text
            print((name.text, val.text))

    try:
        # now pick ranking on industry
        rank = content.find("h4", {"class": "Heading Heading--xl"})
        temp_dict["industry_rank"] = rank.text

        print(temp_dict["industry_rank"])

        # now pick description
        descr = content.find("p", {"class": "Standfirst"})
        temp_dict["descr"] = descr.text

        print(descr.text)
    except:
        temp_dict["industry_rank"], temp_dict["descr"] = "", ""

    return pd.DataFrame(temp_dict, index=[0])

# main run
def run_selenium_srap(df, esg_main_df, folder):

    with open(f"{folder}\\refinitiv_esg\\ind.pkl", 'rb') as f:
        last_ind = pickle.load(f)

    # case where we pick last index, we initialize at zero
    if last_ind>=df.shape[0]-1:
        last_ind=0

    # first we will loop every hour 87 companies to avoid exceed limit from refinitive
    for j in range(0, int((df.shape[0]-last_ind) / 87) + 1):
        for ind in range(last_ind + j * 87, min(df.shape[0],  last_ind + (j + 1) * 87)):

            # we make sur we don't go over dataframe number of rows
            if ind < df.shape[0]:
                # now we loop through company names
                # Now clear selection
                #     button = driver.find_element_by_xpath('//button[@class="SearchInput-clearButton"]')
                #     button.click()
                #     time.sleep(3)

                # Now insert company name
                e = driver.find_element(By.XPATH, '//input[@id="searchInput-1"]')
                e.send_keys(df.loc[ind, "company"])
                time.sleep(2)

                # We click on the enter
                button = driver.find_element(By.XPATH, '//button[@class="SearchInput-searchButton"]')
                button.click()
                time.sleep(2)

                # We can collect ESG score
                cc_html = driver.page_source
                content = BeautifulSoup(cc_html, 'html.parser')

                # We can collect ESG score
                company_df = extract_scores(content)

                # we append to main dataframe
                esg_main_df  = pd.concat([esg_main_df, company_df], axis=0)

                # Now clear selection
                button = driver.find_element(By.XPATH, '//button[@class="SearchInput-clearButton"]')
                button.click()
                # time.sleep(3)
                print("{} company".format(str(ind)))

                # we save every 10 companies
                if ind % 10 == 0:
                    # save the 10 companies
                    save_csv(esg_main_df)

                    # redefine the dataframe as empty
                    esg_main_df = pd.DataFrame([], columns=['Environment', 'Emissions', 'Resource Use', 'Innovation',
                                                            'Social', 'Human Rights', 'Product Responsibility', 'Workforce',
                                                            'Community', 'Governance', 'Management', 'Shareholders', 'CSR Strategy',
                                                            'industry_rank', 'descr', 'main_score'])

                with open(f"{folder}\\refinitiv_esg\\ind.pkl", 'wb') as f:
                    pickle.dump(ind, f)

        # we wait an hour to be able to collect again
        time.sleep(60 * 61)

    # after we looped over all companies, we re-load temp.csv file and define as final df
    esg_main_df = pd.read_csv(f"{folder}\\refinitiv_esg\\temp.csv")

    esg_main_df["Val_date"] = datetime.now()
    esg_main_df.to_csv(f"{folder}\\refinitiv_esg\\esg_score_{datetime.now():%Y-%m-%d %H-%M-%S%z}.csv", index=False)
    with open(f"{folder}\\refinitiv_esg\\ind.pkl", 'wb') as f:
        pickle.dump(0, f)

    # now we save temp as empty csv
    temp = pd.DataFrame(columns=["Environment", "Emissions", "Resource Use", "Innovation", "Social", "Human Rights"
        , "Product Responsibility","Workforce","Community","Governance","Management","Shareholders","CSR Strategy","industry_rank"
        ,"descr","main_score"])

    temp.to_csv(f"{folder}\\refinitiv_esg\\temp.csv", index=False)

def save_csv(df):
    """
    We save in excel
    :return:
    """
    temp = pd.read_csv(r"C:\Users\cytan\Downloads\ESG-scrapper-master\ESG-scrapper-master\refinitiv_esg\temp.csv")

    # we merge with dataframe
    df = pd.concat([df, temp], axis=0)

    # we save again as csv
    df.to_csv(r"C:\Users\cytan\Downloads\ESG-scrapper-master\ESG-scrapper-master\refinitiv_esg\temp.csv", index=False)

if __name__ == "__main__":

    # read the csv file with company names
    folder = os.path.expanduser('~/Downloads\ESG-scrapper-master\ESG-scrapper-master')
    df = pd.read_csv(f"{folder}\\refinitiv_esg\\companies.csv")

    # start by collecting driver chrome
    # chrome_options = Options()
    # chrome_options.add_argument("--headless")
    chrome_driver_path = r'C:\Users\cytan\Downloads\ESG-scrapper-master\ESG-scrapper-master\chromedriver\chromedriver.exe'
    # cService = webdriver.ChromeService(executable_path=chrome_driver_path)
    # driver = webdriver.Chrome(service = cService)
    driver = webdriver.Chrome()
    driver.get("https://www.refinitiv.com/en/sustainable-finance/esg-scores")
    time.sleep(10)

    # start by clicking on cookies
    button = driver.find_element(By.XPATH,'//button[@id="onetrust-accept-btn-handler"]')
    button.click()
    time.sleep(3)

    # ESG final dataframe
    esg_main_df = pd.DataFrame([], columns=['Environment', 'Emissions', 'Resource Use', 'Innovation', 'Social',
                                            'Human Rights', 'Product Responsibility', 'Workforce', 'Community',
                                            'Governance', 'Management', 'Shareholders', 'CSR Strategy',
                                            'industry_rank', 'descr', 'main_score'])

    # run selenium scrap on current page
    run_selenium_srap(df, esg_main_df, folder)

65/100Based on Genting Malaysia Bhd's self-reported FY 2022 data
('Environment', '77')
('Emissions', '82')
('Resource Use', '84')
('Innovation', '0')
('Social', '80')
('Human Rights', '77')
('Product Responsibility', '74')
('Workforce', '81')
('Community', '94')
('Governance', '32')
('Management', '17')
('Shareholders', '77')
('CSR Strategy', '42')
73/346
Genting Malaysia Bhd is a Hotels & Entertainment Services company. Below is the rank of this company out of all the companies in its industry. Get the most comprehensive, detailed, and up to date sector and industry classification available with the  Refinitiv Business Classification
0 company
43/100Based on PARKSON Retail Group Ltd's self-reported FY 2022 data
('Environment', '43')
('Emissions', '37')
('Resource Use', '59')
('Innovation', '0')
('Social', '41')
('Human Rights', '8')
('Product Responsibility', '77')
('Workforce', '60')
('Community', '9')
('Governance', '45')
('Management', '43')
('Shareholders', '57')
('CSR Strategy', 

47/100Based on Walsin Technology Corp's self-reported FY 2022 data
('Environment', '49')
('Emissions', '59')
('Resource Use', '55')
('Innovation', '29')
('Social', '65')
('Human Rights', '43')
('Product Responsibility', '72')
('Workforce', '66')
('Community', '82')
('Governance', '24')
('Management', '11')
('Shareholders', '38')
('CSR Strategy', '66')
79/157
Gallaher Group Ltd is a Electronic Equipment & Parts company. Below is the rank of this company out of all the companies in its industry. Get the most comprehensive, detailed, and up to date sector and industry classification available with the  Refinitiv Business Classification
13 company
73/100Based on Salesforce.Com Inc's self-reported FY 2023 data
('Environment', '76')
('Emissions', '91')
('Resource Use', '97')
('Innovation', '53')
('Social', '76')
('Human Rights', '70')
('Product Responsibility', '34')
('Workforce', '97')
('Community', '98')
('Governance', '69')
('Management', '71')
('Shareholders', '47')
('CSR Strategy', '94'

58/100Based on Monro Inc's self-reported FY 2023 data
('Environment', '32')
('Emissions', '50')
('Resource Use', '32')
('Innovation', '6')
('Social', '50')
('Human Rights', '29')
('Product Responsibility', '53')
('Workforce', '46')
('Community', '63')
('Governance', '81')
('Management', '89')
('Shareholders', '79')
('CSR Strategy', '39')
92/295
GS Home Shopping Inc is a Specialty Retailers company. Below is the rank of this company out of all the companies in its industry. Get the most comprehensive, detailed, and up to date sector and industry classification available with the  Refinitiv Business Classification
26 company
58/100Based on Monro Inc's self-reported FY 2023 data
('Environment', '32')
('Emissions', '50')
('Resource Use', '32')
('Innovation', '6')
('Social', '50')
('Human Rights', '29')
('Product Responsibility', '53')
('Workforce', '46')
('Community', '63')
('Governance', '81')
('Management', '89')
('Shareholders', '79')
('CSR Strategy', '39')
92/295
Automotive Holdings Gr

29/100Based on Arbutus Biopharma Corp's self-reported FY 2022 data
('Environment', '1')
('Emissions', '0')
('Resource Use', '3')
('Innovation', '0')
('Social', '35')
('Human Rights', '0')
('Product Responsibility', '11')
('Workforce', '58')
('Community', '78')
('Governance', '41')
('Management', '36')
('Shareholders', '88')
('CSR Strategy', '0')
286/459
Arbutus Biopharma Corp is a Pharmaceuticals company. Below is the rank of this company out of all the companies in its industry. Get the most comprehensive, detailed, and up to date sector and industry classification available with the  Refinitiv Business Classification
39 company
29/100Based on Arbutus Biopharma Corp's self-reported FY 2022 data
('Environment', '1')
('Emissions', '0')
('Resource Use', '3')
('Innovation', '0')
('Social', '35')
('Human Rights', '0')
('Product Responsibility', '11')
('Workforce', '58')
('Community', '78')
('Governance', '41')
('Management', '36')
('Shareholders', '88')
('CSR Strategy', '0')
286/459
CGTEL 

83/100Based on Tata Consultancy Services Ltd's self-reported FY 2022 data
('Environment', '75')
('Emissions', '99')
('Resource Use', '89')
('Innovation', '53')
('Social', '93')
('Human Rights', '96')
('Product Responsibility', '81')
('Workforce', '100')
('Community', '99')
('Governance', '76')
('Management', '79')
('Shareholders', '48')
('CSR Strategy', '99')
16/932
Tata Consultancy Services Ltd is a Software & IT Services company. Below is the rank of this company out of all the companies in its industry. Get the most comprehensive, detailed, and up to date sector and industry classification available with the  Refinitiv Business Classification
52 company
64/100Based on Lundin Gold Inc's self-reported FY 2022 data
('Environment', '75')
('Emissions', '96')
('Resource Use', '70')
('Innovation', '0')
('Social', '71')
('Human Rights', '72')
('Product Responsibility', '21')
('Workforce', '73')
('Community', '90')
('Governance', '38')
('Management', '25')
('Shareholders', '60')
('CSR Strate

17/100Based on ACE Aviation Holdings Inc's self-reported FY 2010 data
('Environment', '2')
('Emissions', '5')
('Resource Use', '0')
('Innovation', '0')
('Social', '27')
('Human Rights', '0')
('Product Responsibility', '19')
('Workforce', '19')
('Community', '74')
('Governance', '19')
('Management', '25')
('Shareholders', '10')
('CSR Strategy', '0')
116/123
Hexo Corp is a Passenger Transportation Services company. Below is the rank of this company out of all the companies in its industry. Get the most comprehensive, detailed, and up to date sector and industry classification available with the  Refinitiv Business Classification
65 company
61/100Based on China Pacific Insurance Group Co Ltd's self-reported FY 2022 data
('Environment', '69')
('Emissions', '77')
('Resource Use', '84')
('Innovation', '61')
('Social', '32')
('Human Rights', '13')
('Product Responsibility', '34')
('Workforce', '52')
('Community', '20')
('Governance', '93')
('Management', '96')
('Shareholders', '82')
('CSR Str

18/100Based on Luna Innovations Inc's self-reported FY 2022 data
('Environment', '0')
('Emissions', '0')
('Resource Use', '0')
('Innovation', '0')
('Social', '14')
('Human Rights', '0')
('Product Responsibility', '7')
('Workforce', '10')
('Community', '40')
('Governance', '44')
('Management', '51')
('Shareholders', '51')
('CSR Strategy', '0')
581/632
Luna Innovations Inc is a Machinery Tools Heavy Vehicles Trains & Ships company. Below is the rank of this company out of all the companies in its industry. Get the most comprehensive, detailed, and up to date sector and industry classification available with the  Refinitiv Business Classification
78 company
31/100Based on Fulcrum Therapeutics Inc's self-reported FY 2022 data
('Environment', '0')
('Emissions', '0')
('Resource Use', '0')
('Innovation', '0')
('Social', '57')
('Human Rights', '0')
('Product Responsibility', '72')
('Workforce', '71')
('Community', '46')
('Governance', '27')
('Management', '34')
('Shareholders', '20')
('CSR Str